In [10]:
import pandas as pd


# Job Cleaning

In [11]:
# -----------------------
# Utility: clean URLs, emails, phone numbers
# -----------------------

def clean_text(text: str) -> str:
    # Remove web links
    text = re.sub(r"http\S+|www\.\S+", " ", text)
    # Remove email addresses
    text = re.sub(r"\S+@\S+", " ", text)
    # Remove phone numbers (digits, dashes/spaces, length ≥9)
    text = re.sub(r"\+?\d[\d\-\s]{7,}\d", " ", text)
    # Collapse all consecutive whitespace into single spaces
    text = re.sub(r"\s+", " ", text)
    return text.strip()



In [12]:
# -----------------------
# Utility: load & prepare raw job data
# -----------------------

def load_and_prepare_jobs(data_path: str) -> pd.DataFrame: 
    # Read CSV into DataFrame
    df = pd.read_csv(data_path)
    # Reset index to ensure continuous integer IDs
    df = df.reset_index(drop=True)
    # Rename Job-Title column to Job_title
    df = df.rename(columns={'Job-Title': 'Job_title'})
    # Drop any rows missing essential fields
    df = df.dropna(subset=["Job_title", "Description", "Location"]).copy()
    # Create a unique job_id based on the DataFrame index
    df["job_id"] = df.index.astype(int)

    df["title_clean"] = df["Job_title"].apply(clean_text)
    df["desc_clean"] = df["Description"].apply(clean_text)
    df["text"] = df["title_clean"] + " " + df["desc_clean"]
    return df



In [13]:

# Store the job data
df = load_and_prepare_jobs(JOB_DATA_PATH)
df.to_csv(r"C:\Yousuf\DEPI\Technical\Mega Projects\Job_Recommendation_System\data\Processed\cleaned_data\job_cleaned.csv", index=False)
df

,Job_title,Date-Posted,Company,Job-Type,Salary,Location,Description,job_id,title_clean,desc_clean,text
0,"Bachelor of Science, Nursing Instructor Part...",2023-04-25,Red Deer College,Full-time,NaN,"Red Deer, AB",Position Information\n\nPosition Title\n\nBach...,0,"Bachelor of Science, Nursing Instructor Part T...",Position Information Position Title Bachelor o...,"Bachelor of Science, Nursing Instructor Part T..."
1,PHD Graduate Student,2023-04-19,IDOBE Research Group,Full-time,NaN,"Edmonton, AB",The IDOBE (Intelligent Design & Operation for ...,1,PHD Graduate Student,The IDOBE (Intelligent Design & Operation for ...,PHD Graduate Student The IDOBE (Intelligent De...
2,Central Graduate Advisor,2023-04-19,University of Alberta,Full-time,"$47,210–$63,741 a year","Edmonton, AB",FGSR Grad Studies & Rsrch Admi\n\nCompetition ...,2,Central Graduate Advisor,FGSR Grad Studies & Rsrch Admi Competition No....,Central Graduate Advisor FGSR Grad Studies & R...
3,Student/Graduate Application,2023-04-20,Eagle Builders LP,Internship,NaN,"Blackfalds, AB",Eagle Builders welcomes and offers work experi...,3,Student/Graduate Application,Eagle Builders welcomes and offers work experi...,Student/Graduate Application Eagle Builders we...
4,"Advisor, Graduate Student Experience, Haskayne...",2023-04-12,University of Calgary,Full-time,NaN,"Calgary, AB",University of Calgary\nRSS Jump to Headline Ho...,4,"Advisor, Graduate Student Experience, Haskayne...",University of Calgary RSS Jump to Headline Hom...,"Advisor, Graduate Student Experience, Haskayne..."
...,...,...,...,...,...,...,...,...,...,...,...
40056,Bike Centre Volunteer (Fall 2022),NaN,Bike Centre,NaN,NaN,"Waterloo, ON",Service Description: The Bike Centre is a do-i...,40056,Bike Centre Volunteer (Fall 2022),Service Description: The Bike Centre is a do-i...,Bike Centre Volunteer (Fall 2022) Service Desc...
40057,Volunteer Program Coordinator,2023-04-05,The Food Bank of Waterloo Region,Full-time,NaN,"Kitchener, ON",Volunteer Program Coordinator\n\nAdministratio...,40057,Volunteer Program Coordinator,Volunteer Program Coordinator Administration K...,Volunteer Program Coordinator Volunteer Progra...
40058,Volunteer Coordinator,NaN,Alzheimer Society Waterloo Wellington,Part-time,NaN,"Kitchener, ON",About the job\n\nJOB SUMMARY\n\nThis position ...,40058,Volunteer Coordinator,About the job JOB SUMMARY This position manage...,Volunteer Coordinator About the job JOB SUMMAR...
40059,Summer Camp Volunteer,NaN,Little Medical School - Region of Waterloo,Part-time,NaN,"Kitchener, ON",Do you love working and interacting with child...,40059,Summer Camp Volunteer,Do you love working and interacting with child...,Summer Camp Volunteer Do you love working and ...


# User Data Cleaning

In [16]:
user_df = pd.read_csv('C:\Yousuf\DEPI\Technical\Mega Projects\Job_Recommendation_System\data\Raw\skill2vec_50K.csv')
user_df

C:\Users\Yousuf\AppData\Local\Temp\ipykernel_21780\3217808817.py:1: DtypeWarning: Columns (33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,

,HR Executive,screening,selection,Interview,HR,Recruiter,IT Recruiter,Sourcing,recruitment executive,onboarding,...,Unnamed: 950,Unnamed: 951,Unnamed: 952,Unnamed: 953,Unnamed: 954,Unnamed: 955,Unnamed: 956,Unnamed: 957,Unnamed: 958,Unnamed: 959
0,Special Teacher,Teaching,Education,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,consulting,fresher,IT helpdesk,Techincal Troubleshooting,international voice,international BPO,technical support,outsourcing,call center,BBA fresher,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,diploma,machining,cnc m,mould,conventional machines,die making,knowledge,tool,cipet,assembly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Compensation,Benefits,HR Functions,Alm,Payroll,ESS,Core HR,QC,QA,SQL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Storage Administrator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49994,Chief Engineer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49995,Receptionist Activities,Front Desk,front office,front desk executive,front office executive,receptionist,reception,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49996,SQL Queries,Log Analysis,Hardware Networking,People Leadership,Technical Skills,Unix,Oracle,Service Delivery Management,Database Administration,Continuous Improvement,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49997,Quality Analyst,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Produce a list of skills for each profile

In [14]:
def prepare_user_skill_profiles(df: pd.DataFrame):
    # Fill NaNs with empty strings and convert all to string
    df_clean = df.fillna("").astype(str)
    
    # Combine each row's values into a cleaned list of skills
    user_skills = []
    for _, row in df_clean.iterrows():
        skills = [skill.strip().lower() for skill in row if skill.strip()]
        skills = list(dict.fromkeys(skills))  # optional: remove duplicates, preserve order
        user_skills.append(skills)
    
    return user_skills

In [17]:
# Apply the transformation
user_skill_lists = prepare_user_skill_profiles(user_df)

# Show example output
user_skill_lists[:5]

[['special teacher', 'teaching', 'education'],
 ['consulting',
  'fresher',
  'it helpdesk',
  'techincal troubleshooting',
  'international voice',
  'international bpo',
  'technical support',
  'outsourcing',
  'call center',
  'bba fresher',
  'bcom fresher',
  'tech support',
  'voice calling',
  'bpo',
  'sme',
  'bca fresher',
  'mba fresher'],
 ['diploma',
  'machining',
  'cnc m',
  'mould',
  'conventional machines',
  'die making',
  'knowledge',
  'tool',
  'cipet',
  'assembly'],
 ['compensation',
  'benefits',
  'hr functions',
  'alm',
  'payroll',
  'ess',
  'core hr',
  'qc',
  'qa',
  'sql'],
 ['storage administrator']]

In [ ]:
# Store the job data
user_skill_lists.to_csv(r"C:\Yousuf\DEPI\Technical\Mega Projects\Job_Recommendation_System\data\Processed\cleaned_data\skill_list.csv", index=False)